Ejercicio 5: Para los datos del ejercicio 1 de la liga de Futbol. Realizar el ejercicio Python

SE OMITEN RESULTADOS, PORQUE SON LOS MISMOS QUE USANDO R.

A) Usar el algoritmo de selección hacia adelante para seleccionar un modelo de regresión.

In [4]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Cargar los datos
datos = pd.read_csv("Liga_nacional_de_futbol.csv")
print("Datos cargados:")
print(datos.head())
print(f"\nDimensión de los datos: {datos.shape}")


Datos cargados:
    y    x1    x2    x3    x4  x5   x6    x7    x8    x9
0  10  2113  1985  38.9  64.7   4  868  59.7  2205  1917
1  11  2003  2855  38.8  61.3   3  615  55.0  2096  1575
2  11  2957  1737  40.1  60.0  14  914  65.6  1847  2175
3  13  2285  2905  41.6  45.3  -4  957  61.4  1903  2476
4  10  2971  1666  39.2  53.8  15  836  66.1  1457  1866

Dimensión de los datos: (28, 10)


In [7]:
# Función para selección hacia adelante basada en p-valores
def seleccion_adelante_pvalor(data, alpha=0.05):
    """
    Implementa selección hacia adelante basada en p-valores
    """
    variables_disponibles = [col for col in data.columns if col != 'y']
    variables_seleccionadas = []
    
    print("Iniciando selección hacia adelante...")
    print("=" * 50)
    
    # Modelo inicial (solo intercepto) - CORREGIDO
    X = np.ones((len(data), 1))  # Matriz de unos para el intercepto
    y = data['y']
    modelo_actual = sm.OLS(y, X).fit()
    
    paso = 1
    
    while variables_disponibles:
        mejor_pvalor = float('inf')
        mejor_variable = None
        mejor_modelo = None
        
        # Probar cada variable disponible
        for variable in variables_disponibles:
            # Crear matriz de diseño con variables actuales + nueva variable
            if variables_seleccionadas:
                X_temp = data[variables_seleccionadas + [variable]].copy()
            else:
                X_temp = data[[variable]].copy()
            
            # Agregar constante
            X_temp = sm.add_constant(X_temp)
            
            # Ajustar modelo temporal
            modelo_temp = sm.OLS(y, X_temp).fit()
            
            # Obtener p-valor de la nueva variable
            pvalor_nuevo = modelo_temp.pvalues[variable]
            
            # Verificar si es la mejor variable
            if pvalor_nuevo < mejor_pvalor:
                mejor_pvalor = pvalor_nuevo
                mejor_variable = variable
                mejor_modelo = modelo_temp
        
        # Verificar criterio de parada
        if mejor_pvalor > alpha:
            print(f"\nCriterio de parada: Ninguna variable restante tiene p-valor < {alpha}")
            break
        
        # Agregar la mejor variable al modelo
        variables_seleccionadas.append(mejor_variable)
        variables_disponibles.remove(mejor_variable)
        modelo_actual = mejor_modelo
        
        # Mostrar progreso
        print(f"Paso {paso}: Se agregó '{mejor_variable}' (p-valor = {mejor_pvalor:.4f})")
        print(f"   R² ajustado: {modelo_actual.rsquared_adj:.4f}, AIC: {modelo_actual.aic:.2f}")
        
        paso += 1
    
    print("\n" + "=" * 50)
    print("Selección finalizada")
    print(f"Variables seleccionadas: {variables_seleccionadas}")
    print(f"Fórmula final: y ~ {' + '.join(variables_seleccionadas)}")
    print("\nResumen del modelo final:")
    print(modelo_actual.summary())
    
    return modelo_actual, variables_seleccionadas

# Versión simplificada y más robusta
def seleccion_adelante_simple(data, alpha=0.05):
    """
    Implementación simplificada de selección hacia adelante
    """
    variables = [col for col in data.columns if col != 'y']
    seleccionadas = []
    y = data['y']
    
    print("\nMÉTODO SIMPLIFICADO:")
    print("=" * 30)
    
    # Modelo inicial con solo intercepto
    X_const = sm.add_constant(pd.DataFrame())  # DataFrame vacío con constante
    modelo_actual = sm.OLS(y, X_const).fit()
    
    paso = 1
    
    while variables:
        mejor_p = 1.0
        mejor_var = None
        
        for var in variables:
            # Crear matriz con variables seleccionadas + nueva variable
            if seleccionadas:
                X_temp = data[seleccionadas + [var]]
            else:
                X_temp = data[[var]]
            
            X_temp = sm.add_constant(X_temp)
            modelo_temp = sm.OLS(y, X_temp).fit()
            
            p_val = modelo_temp.pvalues[var]
            
            if p_val < mejor_p:
                mejor_p = p_val
                mejor_var = var
                mejor_modelo = modelo_temp
        
        if mejor_p > alpha:
            print(f"Parando: {mejor_var} tiene p-valor {mejor_p:.4f} > {alpha}")
            break
        
        seleccionadas.append(mejor_var)
        variables.remove(mejor_var)
        modelo_actual = mejor_modelo
        
        print(f"Paso {paso}: Agregada {mejor_var} (p={mejor_p:.4f}), R²-adj: {modelo_actual.rsquared_adj:.4f}")
        paso += 1
    
    # Ajustar modelo final con todas las variables seleccionadas
    if seleccionadas:
        X_final = sm.add_constant(data[seleccionadas])
        modelo_final = sm.OLS(y, X_final).fit()
    else:
        modelo_final = modelo_actual  # Solo intercepto
    
    print(f"\nVariables seleccionadas: {seleccionadas}")
    print("Resumen del modelo final:")
    print(modelo_final.summary())
    
    return modelo_final, seleccionadas

# Método alternativo usando solo intercepto inicial
def seleccion_adelante_intercepto(data, alpha=0.05):
    """
    Método que comienza con solo intercepto
    """
    variables = [col for col in data.columns if col != 'y']
    seleccionadas = []
    y = data['y']
    
    print("\nMÉTODO CON INTERCEPTO:")
    print("=" * 30)
    
    paso = 1
    
    while variables:
        mejor_p = 1.0
        mejor_var = None
        
        for var in variables:
            # Crear matriz con constante + variables seleccionadas + nueva variable
            X_temp = data[seleccionadas + [var]] if seleccionadas else data[[var]]
            X_temp = sm.add_constant(X_temp)
            
            modelo_temp = sm.OLS(y, X_temp).fit()
            p_val = modelo_temp.pvalues[var]
            
            if p_val < mejor_p:
                mejor_p = p_val
                mejor_var = var
                mejor_modelo = modelo_temp
        
        if mejor_p > alpha:
            print(f"Parando: ninguna variable tiene p-valor < {alpha}")
            break
        
        seleccionadas.append(mejor_var)
        variables.remove(mejor_var)
        
        print(f"Paso {paso}: {mejor_var} (p={mejor_p:.4f}), R²={mejor_modelo.rsquared:.4f}")
        paso += 1
    
    # Modelo final
    if seleccionadas:
        X_final = sm.add_constant(data[seleccionadas])
        modelo_final = sm.OLS(y, X_final).fit()
    else:
        # Solo intercepto
        X_final = sm.add_constant(pd.DataFrame(index=data.index))
        modelo_final = sm.OLS(y, X_final).fit()
    
    print(f"\nVariables seleccionadas: {seleccionadas}")
    return modelo_final, seleccionadas

# Ejecutar selección hacia adelante
try:
    print("MÉTODO PRINCIPAL:")
    print("=" * 30)
    modelo_final, variables_seleccionadas = seleccion_adelante_pvalor(datos)
except Exception as e:
    print(f"Error en método principal: {e}")


MÉTODO PRINCIPAL:
Iniciando selección hacia adelante...
Paso 1: Se agregó 'x8' (p-valor = 0.0000)
   R² ajustado: 0.5272, AIC: 130.25
Paso 2: Se agregó 'x2' (p-valor = 0.0002)
   R² ajustado: 0.7227, AIC: 116.20
Paso 3: Se agregó 'x7' (p-valor = 0.0378)
   R² ajustado: 0.7596, AIC: 113.06

Criterio de parada: Ninguna variable restante tiene p-valor < 0.05

Selección finalizada
Variables seleccionadas: ['x8', 'x2', 'x7']
Fórmula final: y ~ x8 + x2 + x7

Resumen del modelo final:
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.786
Model:                            OLS   Adj. R-squared:                  0.760
Method:                 Least Squares   F-statistic:                     29.44
Date:                Sun, 07 Sep 2025   Prob (F-statistic):           3.27e-08
Time:                        22:48:10   Log-Likelihood:                -52.532
No. Observations:                  28   AIC

Usar el algoritmo de selección hacia atrás para seleccionar un 
modelo de regresión.

In [8]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Cargar los datos
datos = pd.read_csv("Liga_nacional_de_futbol.csv")
print("Datos cargados:")
print(datos.head())
print(f"\nDimensión de los datos: {datos.shape}")

def seleccion_atras_manual(data, alpha=0.05):
    """
    Implementa selección hacia atrás basada en p-valores
    
    Parameters:
    data: DataFrame con los datos
    alpha: nivel de significancia para mantener variables
    
    Returns:
    modelo_final: modelo statsmodels con variables seleccionadas
    variables_seleccionadas: lista de variables seleccionadas
    """
    
    # Empezar con todas las variables (excluyendo y)
    variables_actuales = [col for col in data.columns if col != 'y']
    y = data['y']
    
    print("Iniciando selección hacia atrás...")
    print("=" * 50)
    print(f"Modelo inicial con {len(variables_actuales)} variables: {variables_actuales}")
    
    # Modelo completo inicial
    X_completo = sm.add_constant(data[variables_actuales])
    modelo_actual = sm.OLS(y, X_completo).fit()
    
    paso = 1
    cambios = True
    
    while cambios and len(variables_actuales) > 0:
        cambios = False
        
        # Obtener p-valores del modelo actual (excluyendo constante)
        pvalores = modelo_actual.pvalues.drop('const', errors='ignore')
        
        # Encontrar la variable con el p-valor más alto
        if len(pvalores) > 0:
            peor_pvalor = pvalores.max()
            peor_variable = pvalores.idxmax()
            
            # Verificar criterio de eliminación
            if peor_pvalor > alpha:
                # Eliminar la variable con p-valor más alto
                variables_actuales.remove(peor_variable)
                
                # Ajustar nuevo modelo
                if len(variables_actuales) > 0:
                    X_nuevo = sm.add_constant(data[variables_actuales])
                    modelo_nuevo = sm.OLS(y, X_nuevo).fit()
                else:
                    # Solo intercepto
                    X_nuevo = sm.add_constant(pd.DataFrame())
                    modelo_nuevo = sm.OLS(y, X_nuevo).fit()
                
                cambios = True
                
                print(f"Paso {paso}: Se eliminó '{peor_variable}' (p-valor = {peor_pvalor:.4f})")
                print(f"   R² ajustado: {modelo_nuevo.rsquared_adj:.4f}, AIC: {modelo_nuevo.aic:.2f}")
                print(f"   Variables restantes: {variables_actuales}")
                
                modelo_actual = modelo_nuevo
                paso += 1
            else:
                print(f"\nCriterio de parada: Todas las variables tienen p-valor <= {alpha}")
                break
        else:
            break
    
    print("\n" + "=" * 50)
    print("Selección finalizada")
    print(f"Variables finales: {variables_actuales}")
    
    if variables_actuales:
        print(f"Fórmula final: y ~ {' + '.join(variables_actuales)}")
    else:
        print("Fórmula final: y ~ 1 (solo intercepto)")
    
    print("\nResumen del modelo final:")
    print(modelo_actual.summary())
    
    return modelo_actual, variables_actuales

def seleccion_atras_detallada(data, alpha=0.05):
    """
    Versión más detallada con información adicional en cada paso
    """
    variables_actuales = [col for col in data.columns if col != 'y']
    y = data['y']
    
    print("Selección hacia atrás - Versión detallada")
    print("=" * 50)
    
    # Modelo completo inicial
    X_completo = sm.add_constant(data[variables_actuales])
    modelo_actual = sm.OLS(y, X_completo).fit()
    
    print(f"Modelo inicial - R² ajustado: {modelo_actual.rsquared_adj:.4f}")
    print("P-valores iniciales:")
    for var, pval in modelo_actual.pvalues.drop('const').items():
        print(f"   {var}: {pval:.4f}")
    print()
    
    paso = 1
    
    while len(variables_actuales) > 0:
        pvalores = modelo_actual.pvalues.drop('const', errors='ignore')
        
        if len(pvalores) == 0:
            break
            
        peor_pvalor = pvalores.max()
        peor_variable = pvalores.idxmax()
        
        if peor_pvalor <= alpha:
            print(f"Todas las variables significativas (p <= {alpha}). Parando.")
            break
        
        # Eliminar variable
        variables_actuales.remove(peor_variable)
        
        # Ajustar nuevo modelo
        if variables_actuales:
            X_nuevo = sm.add_constant(data[variables_actuales])
            modelo_nuevo = sm.OLS(y, X_nuevo).fit()
        else:
            X_nuevo = sm.add_constant(pd.DataFrame(index=data.index))
            modelo_nuevo = sm.OLS(y, X_nuevo).fit()
        
        print(f"--- Paso {paso} ---")
        print(f"Eliminada: {peor_variable} (p-valor: {peor_pvalor:.4f})")
        print(f"R² ajustado: {modelo_nuevo.rsquared_adj:.4f} (cambio: {modelo_nuevo.rsquared_adj - modelo_actual.rsquared_adj:+.4f})")
        print(f"AIC: {modelo_nuevo.aic:.2f} (cambio: {modelo_nuevo.aic - modelo_actual.aic:+.2f})")
        print(f"Variables restantes: {len(variables_actuales)}")
        
        if variables_actuales:
            print("P-valores actuales:")
            for var, pval in modelo_nuevo.pvalues.drop('const').items():
                print(f"   {var}: {pval:.4f}")
        print()
        
        modelo_actual = modelo_nuevo
        paso += 1
    
    print("=" * 50)
    print("PROCESO FINALIZADO")
    print(f"Variables seleccionadas: {variables_actuales}")
    return modelo_actual, variables_actuales

def analizar_modelo_completo(data):
    """
    Análisis del modelo completo para referencia
    """
    y = data['y']
    X_variables = [col for col in data.columns if col != 'y']
    X = sm.add_constant(data[X_variables])
    
    modelo_completo = sm.OLS(y, X).fit()
    
    print("ANÁLISIS DEL MODELO COMPLETO:")
    print("=" * 40)
    print(f"R² ajustado: {modelo_completo.rsquared_adj:.4f}")
    print(f"AIC: {modelo_completo.aic:.2f}")
    print("\nP-valores del modelo completo:")
    
    pvalores = modelo_completo.pvalues.drop('const')
    for var, pval in pvalores.sort_values().items():
        significancia = "***" if pval < 0.001 else "**" if pval < 0.01 else "*" if pval < 0.05 else " (ns)"
        print(f"   {var:4}: {pval:.4f} {significancia}")
    
    return modelo_completo

# Ejecutar el análisis del modelo completo primero
modelo_completo = analizar_modelo_completo(datos)

print("\n" + "=" * 60)
print("EJECUTANDO SELECCIÓN HACIA ATRÁS MANUAL")
print("=" * 60)

# Ejecutar selección hacia atrás
try:
    modelo_final, variables_finales = seleccion_atras_manual(datos, alpha=0.05)
except Exception as e:
    print(f"Error en el método principal: {e}")
    print("Ejecutando versión alternativa...")
    modelo_final, variables_finales = seleccion_atras_detallada(datos, alpha=0.05)

# Comparación final
print("\n" + "=" * 60)
print("COMPARACIÓN FINAL")
print("=" * 60)

print(f"Modelo completo: {len([col for col in datos.columns if col != 'y'])} variables")
print(f"R² ajustado: {modelo_completo.rsquared_adj:.4f}")
print(f"AIC: {modelo_completo.aic:.2f}")

print(f"\nModelo final: {len(variables_finales)} variables")
print(f"R² ajustado: {modelo_final.rsquared_adj:.4f}")
print(f"AIC: {modelo_final.aic:.2f}")

print(f"\nReducción de variables: {len([col for col in datos.columns if col != 'y']) - len(variables_finales)} variables eliminadas")
print(f"Cambio en R² ajustado: {modelo_final.rsquared_adj - modelo_completo.rsquared_adj:+.4f}")
print(f"Cambio en AIC: {modelo_final.aic - modelo_completo.aic:+.2f}")

# Mostrar variables eliminadas vs seleccionadas
variables_todas = [col for col in datos.columns if col != 'y']
variables_eliminadas = list(set(variables_todas) - set(variables_finales))

print(f"\nVariables seleccionadas: {variables_finales}")
print(f"Variables eliminadas: {variables_eliminadas}")

# Análisis de significancia de las variables finales
if variables_finales:
    print("\nSIGNIFICANCIA DE VARIABLES FINALES:")
    print("-" * 40)
    pvalores_finales = modelo_final.pvalues.drop('const', errors='ignore')
    for var, pval in pvalores_finales.sort_values().items():
        sig = "***" if pval < 0.001 else "**" if pval < 0.01 else "*" if pval < 0.05 else " (ns)"
        print(f"{var:4}: p = {pval:.4f} {sig}")

Datos cargados:
    y    x1    x2    x3    x4  x5   x6    x7    x8    x9
0  10  2113  1985  38.9  64.7   4  868  59.7  2205  1917
1  11  2003  2855  38.8  61.3   3  615  55.0  2096  1575
2  11  2957  1737  40.1  60.0  14  914  65.6  1847  2175
3  13  2285  2905  41.6  45.3  -4  957  61.4  1903  2476
4  10  2971  1666  39.2  53.8  15  836  66.1  1457  1866

Dimensión de los datos: (28, 10)
ANÁLISIS DEL MODELO COMPLETO:
R² ajustado: 0.7234
AIC: 120.94

P-valores del modelo completo:
   x2  : 0.0004 ***
   x8  : 0.0738  (ns)
   x9  : 0.2225  (ns)
   x7  : 0.3235  (ns)
   x4  : 0.4533  (ns)
   x6  : 0.6303  (ns)
   x3  : 0.6427  (ns)
   x1  : 0.6903  (ns)
   x5  : 0.9997  (ns)

EJECUTANDO SELECCIÓN HACIA ATRÁS MANUAL
Iniciando selección hacia atrás...
Modelo inicial con 9 variables: ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9']
Paso 1: Se eliminó 'x5' (p-valor = 0.9997)
   R² ajustado: 0.7380, AIC: 118.94
   Variables restantes: ['x1', 'x2', 'x3', 'x4', 'x6', 'x7', 'x8', 'x9']
Pas

C) Usar el algoritmo de regresión por pasos para seleccionar un modelo de regresión.

In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Cargar los datos
datos = pd.read_csv("Liga_nacional_de_futbol.csv")
print("Datos cargados:")
print(datos.head())
print(f"\nDimensión de los datos: {datos.shape}")

def regresion_por_pasos_manual(data, alpha_entrada=0.05, alpha_salida=0.10):
    """
    Implementa regresión por pasos manual (stepwise)
    
    Parameters:
    data: DataFrame con los datos
    alpha_entrada: umbral para entrada de variables
    alpha_salida: umbral para salida de variables
    
    Returns:
    modelo_final: modelo statsmodels con variables seleccionadas
    variables_modelo: lista de variables seleccionadas
    """
    
    # Variables disponibles (excluyendo y)
    variables_disponibles = [col for col in data.columns if col != 'y']
    variables_modelo = []
    y = data['y']
    
    print("Iniciando regresión por pasos manual...")
    print("=" * 60)
    print(f"Criterios: Entrada (p < {alpha_entrada}), Salida (p > {alpha_salida})")
    print()
    
    # Modelo inicial con solo intercepto
    X_const = sm.add_constant(pd.DataFrame(index=data.index))
    modelo_actual = sm.OLS(y, X_const).fit()
    
    paso = 1
    cambios = True
    
    while cambios:
        cambios = False
        
        # FASE 1: ENTRADA (FORWARD)
        if variables_disponibles:
            print(f"--- Paso {paso}: Fase de entrada ---")
            
            mejor_p_entrada = float('inf')
            mejor_var_entrada = None
            mejor_modelo_entrada = None
            
            # Probar cada variable disponible para entrada
            for variable in variables_disponibles:
                # Crear matriz de diseño con variables actuales + nueva variable
                if variables_modelo:
                    X_temp = data[variables_modelo + [variable]].copy()
                else:
                    X_temp = data[[variable]].copy()
                
                X_temp = sm.add_constant(X_temp)
                
                # Ajustar modelo temporal
                modelo_temp = sm.OLS(y, X_temp).fit()
                
                # Obtener p-valor de la nueva variable
                pvalor_nuevo = modelo_temp.pvalues[variable]
                
                # Verificar si es la mejor candidata para entrada
                if pvalor_nuevo < mejor_p_entrada:
                    mejor_p_entrada = pvalor_nuevo
                    mejor_var_entrada = variable
                    mejor_modelo_entrada = modelo_temp
            
            # Verificar criterio de entrada
            if mejor_p_entrada < alpha_entrada:
                # Agregar variable al modelo
                variables_modelo.append(mejor_var_entrada)
                variables_disponibles.remove(mejor_var_entrada)
                modelo_actual = mejor_modelo_entrada
                
                print(f"   ENTRADA: Se agregó '{mejor_var_entrada}' (p = {mejor_p_entrada:.4f})")
                print(f"   R² ajustado: {modelo_actual.rsquared_adj:.4f}, AIC: {modelo_actual.aic:.2f}")
                print(f"   Variables en modelo: {variables_modelo}")
                
                cambios = True
                paso += 1
                continue  # Ir al siguiente paso después de una entrada
            else:
                print(f"   No hay variables para entrada (mejor p = {mejor_p_entrada:.4f} >= {alpha_entrada})")
        
        # FASE 2: SALIDA (BACKWARD)
        if variables_modelo:
            print(f"--- Paso {paso}: Fase de salida ---")
            
            # Obtener p-valores del modelo actual (excluyendo constante)
            pvalores_actuales = modelo_actual.pvalues.drop('const', errors='ignore')
            
            if not pvalores_actuales.empty:
                # Encontrar variable con mayor p-valor
                peor_p_salida = pvalores_actuales.max()
                peor_var_salida = pvalores_actuales.idxmax()
                
                # Verificar criterio de salida
                if peor_p_salida > alpha_salida:
                    # Eliminar variable del modelo
                    variables_modelo.remove(peor_var_salida)
                    variables_disponibles.append(peor_var_salida)
                    
                    # Ajustar nuevo modelo
                    if variables_modelo:
                        X_nuevo = sm.add_constant(data[variables_modelo])
                        modelo_nuevo = sm.OLS(y, X_nuevo).fit()
                    else:
                        X_nuevo = sm.add_constant(pd.DataFrame(index=data.index))
                        modelo_nuevo = sm.OLS(y, X_nuevo).fit()
                    
                    print(f"   SALIDA: Se eliminó '{peor_var_salida}' (p = {peor_p_salida:.4f})")
                    print(f"   R² ajustado: {modelo_nuevo.rsquared_adj:.4f}, AIC: {modelo_nuevo.aic:.2f}")
                    print(f"   Variables en modelo: {variables_modelo}")
                    
                    modelo_actual = modelo_nuevo
                    cambios = True
                    paso += 1
                else:
                    print(f"   Ninguna variable cumple criterio de salida (peor p = {peor_p_salida:.4f} <= {alpha_salida})")
            else:
                print("   No hay variables para evaluar salida")
        
        # Si no hubo cambios en ninguna fase, terminar
        if not cambios:
            if variables_disponibles:
                print(f"No se pueden agregar más variables (p >= {alpha_entrada}) y no se pueden eliminar (p <= {alpha_salida})")
            break
    
    print("\n" + "=" * 60)
    print("Regresión por pasos finalizada")
    print(f"Variables finales: {variables_modelo}")
    
    if variables_modelo:
        print(f"Fórmula final: y ~ {' + '.join(variables_modelo)}")
    else:
        print("Fórmula final: y ~ 1 (solo intercepto)")
    
    # Ajustar modelo final para asegurar consistencia
    if variables_modelo:
        X_final = sm.add_constant(data[variables_modelo])
        modelo_final = sm.OLS(y, X_final).fit()
    else:
        X_final = sm.add_constant(pd.DataFrame(index=data.index))
        modelo_final = sm.OLS(y, X_final).fit()
    
    print("\nResumen del modelo final:")
    print(modelo_final.summary())
    
    return modelo_final, variables_modelo

# Ejecutar el análisis completo
try:
    modelo_final, variables_finales = analizar_proceso_completo(datos)
    
except Exception as e:
    print(f"Error en el proceso principal: {e}")
    print("Ejecutando método stepwise directamente...")
    modelo_final, variables_finales = regresion_por_pasos_manual(datos)


Datos cargados:
    y    x1    x2    x3    x4  x5   x6    x7    x8    x9
0  10  2113  1985  38.9  64.7   4  868  59.7  2205  1917
1  11  2003  2855  38.8  61.3   3  615  55.0  2096  1575
2  11  2957  1737  40.1  60.0  14  914  65.6  1847  2175
3  13  2285  2905  41.6  45.3  -4  957  61.4  1903  2476
4  10  2971  1666  39.2  53.8  15  836  66.1  1457  1866

Dimensión de los datos: (28, 10)
Error en el proceso principal: name 'analizar_proceso_completo' is not defined
Ejecutando método stepwise directamente...
Iniciando regresión por pasos manual...
Criterios: Entrada (p < 0.05), Salida (p > 0.1)

--- Paso 1: Fase de entrada ---
   ENTRADA: Se agregó 'x8' (p = 0.0000)
   R² ajustado: 0.5272, AIC: 130.25
   Variables en modelo: ['x8']
--- Paso 2: Fase de entrada ---
   ENTRADA: Se agregó 'x2' (p = 0.0002)
   R² ajustado: 0.7227, AIC: 116.20
   Variables en modelo: ['x8', 'x2']
--- Paso 3: Fase de entrada ---
   ENTRADA: Se agregó 'x7' (p = 0.0378)
   R² ajustado: 0.7596, AIC: 113.06
   Va

 ****************************************************************************
 D) Comenta los modelos finales en cada uno de los casos anteriores. ¿Cuál tiene 
 más sentido? ¿Cuál modelo usarían?
 ****************************************************************************


 En cada caso se llega al mismo modelo, con las variables x8, x2 y x7. Yo utilizaría
 el algoritmo de selección por pasos, por eficiente y por superar algunas de las
 limitaciones del forward.